# Data node and for-each loop over objects

This example notebook demonstrates moving over a couple of objects:

- Use a data node to retrieve objects from the world via a world query
- Use a loop node in for-each mode to loop over retrieved objects.

In particular, this example covers:

- Behavior tree nodes
    - `Data` with `WorldQuery`
    - `Loop` with `set_for_each_generator()` from data node result
    - Invoke skill with for each value

<div class="alert alert-info">

**Important**

This notebook requires a running Flowstate solution to connect to. To start a solution:

1. Navigate to [flowstate.intrinsic.ai](https://flowstate.intrinsic.ai/) and sign in
   using your registered Flowstate account.

1. Do **one** of the following:
    - Create a new solution:
        1. Click "Create new solution" and choose "From an example".
        1. Select `building_block:building_block_module2`
        1. Click "Create".
    - Or open an existing solution that was created from the `building_block:building_block_module2` example:
        1. Hover over the solution in the list.
        1. Click "Open solution" or "Start solution".

1. Recommended: Keep the browser tab with the Flowstate solution editor open to watch the effect of notebook actions such as running a skill. You can simultaneously interact with the solution through the web UI and the notebook.

</div>

First, connect to your solution and define convenience shortcuts:

In [ ]:
from intrinsic.math.python import data_types
from intrinsic.solutions import deployments
from intrinsic.solutions import behavior_tree as bt
from intrinsic.solutions import worlds
from intrinsic.world.proto import object_world_refs_pb2
from intrinsic.world.proto import object_world_updates_pb2
from intrinsic.math.python import proto_conversion

solution = deployments.connect_to_selected_solution()

executive = solution.executive
resources = solution.resources
products = solution.products
skills = solution.skills
world = solution.world
simulator = solution.simulator
pose_estimators = solution.pose_estimators

enable_gripper = skills.ai.intrinsic.enable_gripper
move_robot = skills.ai.intrinsic.move_robot

## Create Test Environment

The following creates more blocks to later loop over them.

In [ ]:
clone_from = world.get_object("building_block0")
existing_objects = world.list_object_names()
updates = []
building_block_fragment = products.building_block_fragment
for i in range(1, 6):
    object_name = f"building_block{i}"
    if object_name not in existing_objects:
        pose = clone_from.parent_t_this * data_types.Pose3(
            translation=[(i % 2) * -0.1, i // 2 * 0.05, 0]
        )
        world.create_object_from_product(
            product_name=building_block_fragment.name,
            product_metadata=building_block_fragment.metadata,
            scene_object=building_block_fragment.scene_object,
            object_name=object_name,
            parent=clone_from.parent,
            parent_object_t_created_object=pose,
        )

## Create the Behavior Tree

The overall flow is to initialize, then move to the home position, then move over each building block. Next, we create some basic skills.

In [ ]:
init_gripper = enable_gripper(
    clear_faults=True, gripper=resources.picobot_gripper)

move_home = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            joint_position=world.robot.joint_configurations.home,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.MotionType.ANY,
        )
    ],
    arm_part=world.robot,
)

Now specify the data and loop nodes.

The data node uses a world query to retrieve all children of the root object. That's where the building blocks are parentend.

In [ ]:
world_query = (
    bt.WorldQuery()
    .select(
        children_of=object_world_refs_pb2.ObjectReference(
            by_name=object_world_refs_pb2.ObjectReferenceByName(
                object_name="root"
            )
        )
    )
    .filter(name_regex=r"building_block\d+")
    .order(
        by=bt.WorldQuery.OrderCriterion.NAME,
        direction=bt.WorldQuery.OrderDirection.ASCENDING,
    )
)

data_node = bt.Data(
    name="Query Blocks",
    operation=bt.Data.OperationType.CREATE_OR_UPDATE,
    blackboard_key="blocks",
    world_query=world_query,
)

loop_node = bt.Loop()
loop_node.set_for_each_generator(generator_value=data_node.result.items)

block_target = (
    move_robot.intrinsic_proto.world.geometric_constraints.PoseEquality(
        moving_frame=world.picobot_gripper.tool_frame,
        target_frame=loop_node.for_each_value,
        target_frame_offset=data_types.Pose3(
            translation=[0, 0, 0.12],
            rotation=data_types.Rotation3.from_euler_angles([180, 0, 0]),
        ),
    )
)
move_to_block = move_robot(
    motion_segments=[
        move_robot.intrinsic_proto.skills.MotionSegment(
            cartesian_pose=block_target,
            motion_type=move_robot.intrinsic_proto.skills.MotionSegment.JOINT,
            collision_settings=worlds.CollisionSettings.disabled,
        )
    ],
    arm_part=world.robot,
)

loop_node.set_do_child(move_to_block)

my_bt = bt.BehaviorTree(
    root=bt.Sequence([init_gripper, move_home, data_node, loop_node])
)
my_bt.show()

## Run the Behavior Tree

In [ ]:
executive.run(my_bt)